# Intermittent demand forecasting with skforecast

### 下載檔案

In [2]:
%pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 963.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 2.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.2/123.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 5.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests

res = requests.get("https://drive.google.com/uc?export=download&id=1FGpVW76XAtciVi0ZgDBrAr6Gl6xT9YrQ")
with open("entities_series.pkl", "wb") as file:
    file.write(res.content)

In [7]:
import pickle

with open("entities_series.pkl", "rb") as file:
    entities_series = pickle.load(file)

In [10]:
totalFare = entities_series["totalFare"]
totalFare

legId,000d78c93bdee870101b394bd2b91857,00178c9d00fdad81da468d8e3654a2ed,0018c35572fd06c1a73e139dabee4bf6,0024b874e29b78b255058e9e8f625455,0029d051d252fd0d457fb99b4428ca84,00353469710cebd3cc567a3cf90e1ebc,0036af05c94dc6ee28dbed897ab0a702,0038e87dc86af9c4fb968871eb75f146,0039c5b035f61ac6ecd16ea12a705109,003e653f889ba9563c7ccfbc453e91c6,...,ffb1aa95f4a4dc1f62086c931a40e67f,ffb8b5a9963a3012c5e9976432132f24,ffc08740ee2ce44f5cd05409e3fc0a2a,ffcec16c7e8ec6f0961b54d2f69517e3,ffdc1437c6bb3dae9e07d20a842be906,ffdcc278a9ee3896c233b95f052faaa0,ffde7e030d8d43bd32f3d79da69194c9,ffe311a18192b112695ce8d1c25b52c8,ffef533deef77db02037871790907d77,fff4d4d9dd1b3b43c3e93450d3f28984
searchDate,,,,,,,,,,,,,,,,,,,,,
2022-04-16,NaN,NaN,NaN,448.6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-17,NaN,NaN,483.6,398.6,NaN,NaN,368.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,388.59,NaN,NaN,NaN,318.60
2022-04-18,NaN,NaN,483.6,498.6,NaN,NaN,368.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,318.60
2022-04-19,NaN,NaN,483.6,498.6,198.59,NaN,368.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,478.59,NaN,NaN,NaN,468.61
2022-04-20,NaN,NaN,NaN,498.6,NaN,NaN,358.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,358.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,508.6,508.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308.6,NaN
2022-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,608.6,508.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308.6,NaN
2022-10-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,608.6,508.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308.6,NaN


連結在這邊: 
[Intermittent demand forecasting with skforecast](https://www.cienciadedatos.net/documentos/py48-intermittent-demand-forecasting.html)

我目前還找不到他們有提供 ipynb 檔案，可能只能先複製貼上了

因為 skforecast 預設 exogenous variables 是不同 timeseries 間共用的，所以沒辦法輸入機票的個別資訊

嗯... 我現在才想起來他不能輸入個別的 exogenous variable ... 所以下面會有一些改動

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
# Create forecaster
# ==============================================================================
# 每次訓練一個 id （也就是每次訓練一張機票的資料）
# 開 warm_start 讓模型記得上次訓練的成果
# 沒有 warm_start 選項的可以在一次訓練後把模型的參數取出來，當下次訓練的 initial guess
forest = RandomForestRegressor(warm_start=True)
forecaster = ForecasterAutoreg(
    regressor = forest,
    lags = 24,
)
forecaster